In [1]:
import transformers
import torch
import tensorflow
import pandas as pd
import string
from collections import defaultdict
from tqdm import tqdm

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

from transformers.generation.configuration_utils import GenerationConfig
from sentence_transformers import SentenceTransformer
import re
import numpy as np
from random import random
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

2024-07-29 19:12:57.727052: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 19:12:57.804941: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-29 19:12:59.044479: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd
df = pd.read_csv("paws.csv")
df.shape

(500, 4)

In [3]:
df['input'] = 'Sentence1- ' + df['sentence1'] + '\nSentence2- ' + df['sentence2']

In [4]:
model_path = "/projects/rtelidevara/shared-lib/rtelidevara/llama3-instruct/"  

tokenizer = AutoTokenizer.from_pretrained(model_path)
generation_config = GenerationConfig.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
zero_shot_system_prompt = """
You are an advanced language model designed to analyze pairs of sentences to determine if they are paraphrases of each other. Your task is to give a score of 0 or 1 based on the criteria provided below:
Input format:
Sentence1-..
Sentence2-...

Scoring Criteria:
Score of 1: The two sentences convey the same meaning or very similar meanings. Despite potential differences in wording, structure, or lexical choices, both sentences should represent the same underlying message or information.
Minor differences in phrasing, synonyms, or sentence construction should not affect the overall similarity in meaning.

Score of 0:The two sentences do not convey the same meaning. Even if the sentences share many words or have high lexical overlap, they should be scored as 0 if they represent different meanings or provide different information.
Any significant difference in the message, intent, or factual content between the sentences should result in a score of 0.

Following these criterion, analyse the statements and output just the score, don't output any reasoning.
"""

In [7]:
def llama_v3_prompt(messages, system_prompt):
    B_INST, E_INST = "<|start_header_id|>{user}<|end_header_id|>", "<|eot_id|>"
    B_SYS, E_SYS = "<|start_header_id|>{system}<|end_header_id|>", "<|eot_id|>"
    BOS, EOS = "<|begin_of_text|>", "<|end_of_text|>"
    ASSISTANT = "<|start_header_id|>assistant<|end_header_id|>"
    DEFAULT_SYSTEM_PROMPT = system_prompt

    if messages[0]["role"] == "system":
        first = messages[0]
        messages = messages[1:]
    else:
        first = {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT.strip(),
            }
    first_list = [BOS + B_SYS + "\n\n" + first["content"].strip() + E_SYS]
    messages_list = []
    count = 0
    for prompt, answer in zip(messages[::2], messages[1::2]):
        if count != 0:
          messages_list.append(f"{BOS}{B_INST}\n\n{(prompt['content']).strip()}{E_INST}{ASSISTANT}\n\n{(answer['content']).strip()}{E_INST}{EOS}")
        else:
          messages_list.append(f"{B_INST}\n\n{(prompt['content']).strip()}{E_INST}{ASSISTANT}\n\n{(answer['content']).strip()}{E_INST}{EOS}")
        count += 1

    if len(messages) != 1:
      messages_list.append(f"{BOS}{B_INST}\n\n{(messages[-1]['content']).strip()}{E_INST}{ASSISTANT}")
    else:
      messages_list.append(f"{B_INST}\n\n{(messages[-1]['content']).strip()}{E_INST}{ASSISTANT}")

    return "".join(first_list + messages_list)

In [8]:
def get_prompt(dialogs, system_prompt):
    temp = llama_v3_prompt(dialogs, system_prompt)
    prompt = f"""{temp}"""
    return prompt

In [9]:
def get_response(prompt):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=800)
    result = pipe(f"{prompt}")
    generated_text = result[0]['generated_text']

    if generated_text.startswith(prompt):
        final_response = generated_text[len(prompt):]
    else:
        final_response = generated_text 

    final_response = final_response.strip()
    return final_response

In [10]:
predicted_labels = []

# Iterate over each row in the DataFrame and get the response from the LLM
for index, row in df.iterrows():
    text = row['input']
    
    dialogs = [{"role": "user",
                "content": text}]
    prompt = get_prompt(dialogs, zero_shot_system_prompt)
    response = get_response(prompt)
    # print(text)
    # print("--")
    # print(response)
    predicted_labels.append(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [11]:
def convert_to_int(s):
    if 'Score: ' in s:
        return int(s.split(': ')[1])
    else:
        return int(s)

In [12]:
predicted_labels = [convert_to_int(x) for x in predicted_labels if x in ['0', '1', 'Score: 0', 'Score: 1']]

zero_shot_results_df = df.copy()
zero_shot_results_df['predicted_label'] = predicted_labels

In [13]:
mismatch_count = (zero_shot_results_df['label'] != zero_shot_results_df['predicted_label']).sum()
print(mismatch_count)

267


In [14]:
few_shot_system_prompt = """
You are an advanced language model designed to analyze pairs of sentences to determine if they are paraphrases of each other. Your task is to give a score of 0 or 1 based on the criteria provided below:
Input format:
Sentence1-..
Sentence2-...

Scoring Criteria:
Score of 1: The two sentences convey the same meaning or very similar meanings. Despite potential differences in wording, structure, or lexical choices, both sentences should represent the same underlying message or information.
Minor differences in phrasing, synonyms, or sentence construction should not affect the overall similarity in meaning.

Score of 0:The two sentences do not convey the same meaning. Even if the sentences share many words or have high lexical overlap, they should be scored as 0 if they represent different meanings or provide different information.
Any significant difference in the message, intent, or factual content between the sentences should result in a score of 0.

Here are some examples:
1. Sentence1- To get there , take the Marine Drive from the Lions Gate Bridge to the west , past the Horseshoe Bay , Lighthouse Park and continue on to the 7100 Marine Drive block.
Sentence2- To get there , take Marine Drive west from the Lions Gate Bridge past Lighthouse Park to Horseshoe Bay and then continue on to 7100 Block Marine Drive .
Score: 0
    
2. Sentence1- The Meads of Asphodel is a British black metal band with pronounced medieval influences , eastern , punk and progressive metal from Hertfordshire in the United Kingdom .
Sentence2- The Meads of Asphodel are a British black metal band with pronounced medieval , Eastern , Punk , and Progressive Metal influences from Hertfordshire in the United Kingdom .
Score: 1

3. Sentence1- The two above controls are active : they are either boolean or not .
Sentence2- The two controls mentioned above are boolean : they are either active or not active .
Score: 0

4. Sentence1- Particularly dense erosion occurred in warmer melting periods during the ice ages , when tundra climate reigned and the soil was not protected by a strong cover of vegetation .
Sentence2- Particularly dense erosion occurred in warmer melting periods during the ice ages , when the climate of the tundra prevailed and the soil was not protected by a strong vegetation .
Score: 1

5. Sentence1- The family first moved to Halifax , later moving to Virginia with his father in May 1750 .
Sentence2- The family moved to Virginia first , later they moved to Halifax with his father in May 1750 .
Score: 0

6. Sentence1- After their death in June 1938 , in 1940 he married George Graham , daughter of Jessie .
Sentence2- After her death in June 1938 he married secondly George Graham , daughter of Jessie , in 1940 .
Score: 1
    
Understand the patterns from the examples, and following these criterion, analyse the statements and output just the score, don't output any reasoning.
"""

In [15]:
predicted_labels = []

# Iterate over each row in the DataFrame and get the response from the LLM
for index, row in df.iterrows():
    text = row['input']
    
    dialogs = [{"role": "user",
                "content": text}]
    prompt = get_prompt(dialogs, few_shot_system_prompt)
    response = get_response(prompt)
    # print(text)
    # print("--")
    # print(response)
    predicted_labels.append(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [16]:
print(predicted_labels)

['1', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', 'Score: 0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '1', '1', '0', '1', 'Score: 0', '1', '1', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', 'Score: 1', '1', '1', '1', '1', '1', '1', '1', '0', '0', 'Score: 1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '1', 'Score: 0', '0', '1', '0', '1', '1', '1', '1', '0', '0', 'Score: 0', '1', '0', '1', '1', '0', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '1', '0

In [17]:
predicted_labels = [convert_to_int(x) for x in predicted_labels if x in ['0', '1', 'Score: 0', 'Score: 1']]

few_shot_results_df = df.copy()
few_shot_results_df['predicted_label'] = predicted_labels

In [18]:
mismatch_count = (few_shot_results_df['label'] != few_shot_results_df['predicted_label']).sum()
print(mismatch_count)

186
